In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import xlrd
import tkinter as tk
from tkinter import filedialog
import math

In [2]:
# Intialize GUI interface to select file
root = tk.Tk()
root.winfo_toplevel().title("Select csv files")
root.withdraw()
# Grab Protocol File. This holds information about each trial
print("Please select Protocol File")
prot_path = filedialog.askopenfilename()
prot_path

Please select Protocol File


'V:/R01 - W4K/2_Shaker project/Devices Evaluated/W4K - Shaker Table Pilot Protocol Testing_v8 - Copy.xlsx'

In [3]:
# Grab other information about file:
device = int(input("Which device sheet do you want to read in? "))
trial_number = int(input("Which trial would you like to process? ")) - 1

Which device sheet do you want to read in?0
Which trial would you like to process? 2


In [4]:
# This block creates a function that reads in the protocol sheet
def protocol_reader(path, device) :
    # Since the the file is an excel file with multiple worksheets read it in as an excel object
    all_prot = pd.ExcelFile(path)
    # Reads in a particular sheet from the workbook
    g_prot = all_prot.parse(sheet_name = all_prot.sheet_names[0])
    # Clean up the data frame. The first column is empty so drop it
    g_prot.drop(columns=g_prot.columns[[0,-1]], inplace=True)
    # The header of the data frame is empty. Replace with the first row.
    g_prot.columns = g_prot.iloc[0]
    g_prot = g_prot.iloc[1:]
    # Drop Rows with NaN values
    g_prot.dropna(axis=0, how="all", inplace=True)
    
    return g_prot

In [5]:
garmin_prot = protocol_reader(prot_path, device)

In [6]:
garmin_prot.iloc[:, 5:16]

,Trial #,Round #,Device 1 Serial Number,Device 1 Pacement,Device 2 Serial Number,Device 2 Placement,Device 3 Serial Number,Device 3 Placement,Device 4 Serial Number,Device 4 Placement,Device 5 Serial Number
1,1,1,44,1,97,2,37,3,36,4,113
2,2,1,77,1,84,2,210,3,39,4,20
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,2,77,1,84,2,210,3,39,4,20
5,4,2,44,1,97,2,37,3,36,4,113
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5,3,44,1,97,2,37,3,36,4,113
8,6,3,77,1,84,2,210,3,39,4,20
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,7,4,77,1,84,2,210,3,39,4,20


In [7]:
garmin_prot.iloc[:, 17:23]

,Device 1 Serial Number,Device 1 Paired Phone,Device 2 Placement,Device 2 Serial Number,Device 2 Paired Phone,Device 2 Placement
1,2458,Michelle,6,2428,James,7
2,2458,Michelle,6,2428,James,7
3,NaN,NaN,NaN,NaN,NaN,NaN
4,2342,Morgan,6,2374,Glenn,7
5,2342,Morgan,6,2374,Glenn,7
6,NaN,NaN,NaN,NaN,NaN,NaN
7,2454,W4K Phone 1,6,2390,W4K Phone 3,7
8,2454,W4K Phone 1,6,2390,W4K Phone 3,7
9,NaN,NaN,NaN,NaN,NaN,NaN
10,2384,James,6,2385,W4K Phone 2,7


In [8]:
trials = {}
proxy_count = 1
acti_count = 1
for i in range(7):
    # Grab files of trial:
    if proxy_count <= 2 :
        print(f"Select proxy {proxy_count}")
        trials["proxy" + str(proxy_count)] = filedialog.askopenfilename()
        proxy_count += 1
    else :
        print(f"Select actigraph {acti_count}")
        trials["acti" + str(acti_count)] = filedialog.askopenfilename()
        acti_count += 1
    
trials 

Select proxy 1
Select proxy 2
Select actigraph 1
Select actigraph 2
Select actigraph 3
Select actigraph 4
Select actigraph 5


{'proxy1': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/G2458_V2.csv',
 'proxy2': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/G2428_V2.csv',
 'acti1': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/AG77_clean_trial_2_jw.xlsx',
 'acti2': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/AG84_clean_trial_2_jw.xlsx',
 'acti3': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/AG210_clean_trial_2_jw.xlsx',
 'acti4': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/AG39_clean_trial_2_jw.xlsx',
 'acti5': 'C:/Users/Nick/Watch_Extraction/Shaker_Table/Data/Garmin/Trial 02/AG20_clean_trial_2_jw.xlsx'}

In [9]:
# Need to extract meta info about trial (trial number, round number, device ids)
# Returns a tuple of:
# Trial Number, Round Number, Dictionary of Actigraph IDs, Dictionary of Proxy IDs
def meta_extract(data, trial, prox_num, acti_num):
    # initailize data structures to hold device ids
    proxy_id = {}
    acti_id = {}
    # Extract trial and round from dataframe
    trial_num = data.iloc[trial, 5]
    round_num = data.iloc[trial, 6]
    # Hard code the indices where the ids start for each type of device
    acti_index = 7
    prox_index = 17
    # Collect all proxy ids
    for i in range(prox_num):
        proxy_id["proxy" + str(i+1)] = data.iloc[trial, prox_index]
        prox_index += 3
    # Collect all actigraph ids
    for i in range(acti_num):
        acti_id["acti" + str(i+1)] = data.iloc[trial, acti_index]
        acti_index +=2
        
    return trial_num, round_num, acti_id, proxy_id
    

In [10]:
trial_meta = meta_extract(garmin_prot, trial_number, 2, 5)

In [11]:
trial_meta

(2,
 1,
 {'acti1': 77, 'acti2': 84, 'acti3': 210, 'acti4': 39, 'acti5': 20},
 {'proxy1': 2458, 'proxy2': 2428})

In [12]:
# This function  takes as input the protocol sheet, a trial number and device (actigraph or proxy)
# This function returns a dictonary of time trial start times

def trial_times(data, trial_num):
    acti_time = {}
    proxy_time = {}
    # This is the index where the first actigraph trial start time is listed
    acti_index = -18
    prox_index = acti_index + 1
    # Grab date of the trial this is listed in column 3
    date = data.iloc[trial_num, 3]
    
    for i in range(7):
        # This gets the start time from the protocol sheet
        acti_start = data.iloc[trial_num, acti_index]
        prox_start = data.iloc[trial_num, prox_index]
        # This adds the date and time to the dictionaries
        acti_time['setting_' + str(7-i)] = datetime.combine(date, acti_start) 
        proxy_time['setting_' + str(7-i)] = datetime.combine(date, prox_start)
        
        # Increment both the indices
        acti_index += 2
        prox_index += 2
        
    return acti_time, proxy_time


In [13]:
# A tuple of dictonaries 
# actigraph startimes in position 0
# Proxy statimes in position 1
start_times = trial_times(garmin_prot, trial_number)

In [14]:
# Actigraph start times
start_times[0]

{'setting_7': datetime.datetime(2022, 3, 9, 11, 54, 45),
 'setting_6': datetime.datetime(2022, 3, 9, 11, 56, 45),
 'setting_5': datetime.datetime(2022, 3, 9, 11, 58, 45),
 'setting_4': datetime.datetime(2022, 3, 9, 12, 0, 45),
 'setting_3': datetime.datetime(2022, 3, 9, 12, 2, 45),
 'setting_2': datetime.datetime(2022, 3, 9, 12, 4, 45),
 'setting_1': datetime.datetime(2022, 3, 9, 12, 6, 45)}

In [15]:
# Proxy Start times
start_times[1]

{'setting_7': datetime.datetime(2022, 3, 9, 11, 54, 45),
 'setting_6': datetime.datetime(2022, 3, 9, 11, 56, 45),
 'setting_5': datetime.datetime(2022, 3, 9, 11, 58, 45),
 'setting_4': datetime.datetime(2022, 3, 9, 12, 0, 45),
 'setting_3': datetime.datetime(2022, 3, 9, 12, 2, 45),
 'setting_2': datetime.datetime(2022, 3, 9, 12, 4, 45),
 'setting_1': datetime.datetime(2022, 3, 9, 12, 6, 45)}

In [16]:
# This block will define a function that takes as input a dictionary of file paths
# This function returns a dictionary of dataframes
def device_reader(files, device) :
    prox_count = 1
    acti_count = 1
    data = {}
    for path in files :
        # READ IN GARMIN
        if path[:5] == "proxy" and device == 0:
            data["proxy" + str(prox_count)] = pd.read_csv(files[path])
            # data["proxy" + str(prox_count)] = pd.read_excel(files[path])
            
            prox_count += 1
        # READ IN APPLE
        elif path[:5] == "proxy" and device == 1:
            # Need to check the delimeter of the file. If the delimiter is True it's comma delimited
            # intitally assume comma
            delim = True
            # open file and peek at first line
            check = open(files[path])
            line = check.readline()
            # determine if it is pipe or comma delimited
            for character in line :
                if character == '|' :
                    delim = False
                    break
                elif character == ',' :
                    break
            check.close()
            # Now read in the file based upon delimiter
            if delim :
                data["proxy" + str(prox_count)] = pd.read_csv(files[path])
            else :
                data["proxy" + str(prox_count)] = pd.read_csv(files[path], delimiter='|')
            prox_count += 1  
        # READ IN ACTIGRAPH
        else:
            #data["acti" + str(acti_count)] = pd.read_csv(files[path], skiprows=10)
            data["acti" + str(acti_count)] = pd.read_excel(files[path], skiprows=10)
            acti_count += 1
    return data

In [17]:
test = device_reader(trials, device)

In [24]:
test.keys()

dict_keys(['proxy1', 'proxy2', 'acti1', 'acti2', 'acti3', 'acti4', 'acti5'])

In [18]:
test['proxy1']

,Time,Reading #,X,Y,Z
0,2022-03-09 11:11:18,1,NaN,NaN,NaN
1,2022-03-09 11:11:19,1,-204.0,-384.0,-876.0
2,2022-03-09 11:11:19,2,-128.0,-456.0,-1004.0
3,2022-03-09 11:11:19,3,388.0,-616.0,-888.0
4,2022-03-09 11:11:19,4,592.0,-336.0,-800.0
...,...,...,...,...,...
89596,2022-03-09 12:11:03,21,-32.0,-32.0,-1036.0
89597,2022-03-09 12:11:03,22,-32.0,-32.0,-1036.0
89598,2022-03-09 12:11:03,23,-32.0,-48.0,-1040.0
89599,2022-03-09 12:11:03,24,-40.0,-32.0,-1028.0


In [19]:
# This function converts the garmin, apple, and actigraph timestamps into python datetimes. This allows 
# comparison of times
# To inputs data and device
# data is a dictionary of dataframes, corresponds to data from 7 devices (2 proxy and 5 actigraph)
# device specifies proxy (apple watch or garmin)
def simple(aTime) :
    aTime.microsecond = 0
    
def proxy_time_convert(data, device):
    for table in data :
        if table[:5] == "proxy" :
            if device == 0:
                # Converts garmin timestamp to python datetime
                data[table]["Time"] = pd.to_datetime(data[table]["Time"])
                data[table]["Time"] = data[table]["Time"].apply(lambda x: x.replace(microsecond=0) )
                
                
            else :
                # Rename time and acceleration columns
                data[table].rename(columns={'loggingTime(txt)': 'Time', 'accelerometerTimestamp_sinceReboot(s)': 'Reading #',
                                            'accelerometerAccelerationX(G)': 'X', 'accelerometerAccelerationY(G)': 'Y',
                                            "accelerometerAccelerationZ(G)": 'Z'}, inplace = True)
                # Convert timestamp into datetime
                data[table]["Time"] = data[table]["Time"].apply(lambda x: datetime.fromisoformat(x[:-6]))
                data[table]["Time"] = pd.to_datetime(data[table]["Time"])
                data[table]["Reading #"] = data[table]["Reading #"].apply(lambda x: np.nan)
                data[table] = data[table].iloc[:,:5]
                
        else :
            # Converts actigraph timestamp into python datetime
            # data[table]['Timestamp'] = data[table]["Timestamp"].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M:%S.%f'))
            data[table] = data[table].rename(columns = {'Timestamp':'Time'})


In [20]:
proxy_time_convert(test, device)

In [21]:
test["proxy1"]

,Time,Reading #,X,Y,Z
0,2022-03-09 11:11:18,1,NaN,NaN,NaN
1,2022-03-09 11:11:19,1,-204.0,-384.0,-876.0
2,2022-03-09 11:11:19,2,-128.0,-456.0,-1004.0
3,2022-03-09 11:11:19,3,388.0,-616.0,-888.0
4,2022-03-09 11:11:19,4,592.0,-336.0,-800.0
...,...,...,...,...,...
89596,2022-03-09 12:11:03,21,-32.0,-32.0,-1036.0
89597,2022-03-09 12:11:03,22,-32.0,-32.0,-1036.0
89598,2022-03-09 12:11:03,23,-32.0,-48.0,-1040.0
89599,2022-03-09 12:11:03,24,-40.0,-32.0,-1028.0


In [22]:
test['proxy2']

,Time,Reading #,X,Y,Z
0,2022-03-09 11:13:16,1,204,-744,-680
1,2022-03-09 11:13:16,2,312,-688,-644
2,2022-03-09 11:13:16,3,-260,-1264,-348
3,2022-03-09 11:13:16,4,-40,-1176,-640
4,2022-03-09 11:13:16,5,1544,400,-740
...,...,...,...,...,...
87170,2022-03-09 12:11:22,21,0,8,-972
87171,2022-03-09 12:11:22,22,4,0,-972
87172,2022-03-09 12:11:22,23,0,4,-976
87173,2022-03-09 12:11:22,24,-8,4,-972


In [23]:
test["acti1"]

,Time,Accelerometer X,Accelerometer Y,Accelerometer Z
0,2022-03-09 11:54:29.534,0.004,0.008,-1.023
1,2022-03-09 11:54:29.567,0.004,0.008,-1.023
2,2022-03-09 11:54:29.600,0.004,0.008,-1.023
3,2022-03-09 11:54:29.634,0.004,0.008,-1.023
4,2022-03-09 11:54:29.667,0.004,0.008,-1.023
...,...,...,...,...
117898,2022-03-09 12:59:59.834,-0.012,0.020,-1.023
117899,2022-03-09 12:59:59.867,-0.012,0.020,-1.023
117900,2022-03-09 12:59:59.900,-0.012,0.020,-1.023
117901,2022-03-09 12:59:59.934,-0.012,0.020,-1.023


In [24]:
# This function reads through each data frame and grabs the minutes needed.
# To inputs devices which is a dictionary of dataframes and
# Times which is a tuple of dictionaries of the start times
# This function returns a dictionary of dataframes that have been filtered to only contain data during the trial
def minute_extractor(devices, times, meta_data) :
    # iterate through each device
    filtered = {}
    speeds = ["0.6 Hz", "1.0 Hz", "1.5 Hz", "1.9 Hz", "2.4 Hz", "2.8 Hz", "3.2 Hz"]
    
    for data in devices:
        # used to keep track of location in dataframe
        index = 0
        # Store the  dataframes in a list.
        readings = None
        # Since proxy and acti can have different start times need to check which device
        trials = None
        if data[:5] == "proxy":
            # Get dictionary of start times
            trials = times[1]
            # Get Device ID
            dev_id = meta_data[3][data]
        else :
            trials = times[0]
            dev_id = meta_data[2][data]
        # intialize speed of trial
        speed_index = 6
        # Iterates the start times
        for trial in trials:
            start = trials[trial]
            # Each trial is 2 minutes long. To get the end time add 2 minutes.
            end = start + timedelta(minutes=2)
            # Filter the device data to be between the start and end time
            temp = devices[data].loc[(devices[data]['Time'] >= start) & (devices[data]["Time"] < end)]
            # Next I will insert the speed into the dataframe
            temp.insert(0, "Speed", speeds[speed_index], True)
            speed_index -= 1
            # Insert include column
            temp.insert(2, "Include", 0, True)
            temp.loc[ (temp.Time >= (start + timedelta(seconds=30))) & (temp.Time < (end - timedelta(seconds=30))), "Include" ] = 1
            # Add filtered data to new dataframe
            if readings is None :
                readings = temp
            else :
                readings = pd.concat([readings, temp], axis=0)
                
        # Insert rest of trial meta data into dataframe
        readings.insert(0, "Trial Number", meta_data[0], True )
        readings.insert(1, "Round Number", meta_data[1], True )
        readings.insert(3, "ID", dev_id, True)
        # Check if proxy has heartrate data. If it does remove it
        if 'Heart Rate' in readings.columns :
            readings.drop(columns='Heart Rate', inplace=True)
        
        filtered[data] = readings
         
    return filtered
            
    

In [25]:
trial = minute_extractor(test, start_times, trial_meta)

C:\Users\Nick\Anaconda3\envs\watch_extract\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [26]:
trial

{'proxy1':        Trial Number  Round Number   Speed    ID                Time  Include  \
 65151             2             1  3.2 Hz  2458 2022-03-09 11:54:45        0   
 65152             2             1  3.2 Hz  2458 2022-03-09 11:54:45        0   
 65153             2             1  3.2 Hz  2458 2022-03-09 11:54:45        0   
 65154             2             1  3.2 Hz  2458 2022-03-09 11:54:45        0   
 65155             2             1  3.2 Hz  2458 2022-03-09 11:54:45        0   
 ...             ...           ...     ...   ...                 ...      ...   
 86121             2             1  0.6 Hz  2458 2022-03-09 12:08:44        0   
 86122             2             1  0.6 Hz  2458 2022-03-09 12:08:44        0   
 86123             2             1  0.6 Hz  2458 2022-03-09 12:08:44        0   
 86124             2             1  0.6 Hz  2458 2022-03-09 12:08:44        0   
 86125             2             1  0.6 Hz  2458 2022-03-09 12:08:44        0   
 
        Reading 

In [27]:
# This function takes as input 2 data frames an actigraph and a proxy
# It then converts them to numpy arrays and iterates through them, comparing the times of each reading.
# It returns 1 data frame with readings from both dataframe, ordered by time.
def data_compare(actigraph, proxy):
    # Get shape of dataframes
    a_rows, a_columns = actigraph.shape
    p_rows, p_columns = proxy.shape
    max_rows = max(a_rows, p_rows)
    output = np.empty([max_rows, a_columns + p_columns], dtype="O")
    # Convert to numpy arrays
    n_acti = actigraph.to_numpy()
    n_prox = proxy.to_numpy()
    # intialize variables before loop
    acti = None
    prox = None
    p_row = 0
    # Iterate through both devices aligning data.
    for row in range(max_rows):
        out_row = []
        # Get next actigraph reading
        if row < a_rows :
            acti = n_acti[row]
        # Get proxy reading
        if p_row < p_rows :
            prox = n_prox[p_row]
        # Add actigraph to row
        for item in acti :
            out_row.append(item)
        # Check if proxy row should be added
        if acti[4] >= prox[4] and p_row < p_rows :
            # add proxy reading
            for item in prox :
                out_row.append(item)
            # Increment row in proxy
            p_row += 1
        else :
            # If no prox reading fill last 9 columns with NaNs
            for i in range(p_columns):
                out_row.append(np.nan)
        # Add row to output array
        # print(out_row)
        output[row, :] = out_row
    # Get new columns
    new_columns = ['Trial Number', 'Round Number', 'Speed', 'Actigraph ID', 'Actigraph Time','Include', 'Actigraph X', 
                   'Actigraph Y', 'Actigraph Z', 'Trial Number2', 'Round Number2', 'Speed2', 'Proxy ID', 'Proxy Time',
                  "Include2", "Reading #", "Proxy X", "Proxy Y", "Proxy Z"]
    # print(output)
    # print(output.shape)
    # print(len(new_columns))
    output_df = pd.DataFrame(output, columns=new_columns)
    output_df.drop(columns=["Trial Number2", "Round Number2", "Speed2","Include2"], inplace=True)
    return output_df

In [28]:
# This function aligns every actigraph's data from the trial with every proxy(garmin/apple) from the trial
# Every actigraph device will be compared to each proxy 
# The input to this function is a dictionary of the filtered trial data.
# There are 2 outputs:
# 1 a dataframe that contains all possible actigraphs alinged with all possible proxy devices(This is later stored as a csv)
# 2 a dictionary of dataframes that contain 1 actigraph alinged with 1 device(Used late to get inner minute)
def device_aligner(devices):
    # This loop iterates through the dictionary and finds each actigraph device.
    aligned = None
    pair_count = 1
    pairs = {}
    for a_data in devices :
        # Checks if the device is an actigraph
        # print(a_data)
        if a_data[:-1] == "acti" : 
            # This loop iterates through the dictionary and finds each proxy device(apple/garmin)
            for p_data in devices :
                if p_data[:-1] == "proxy":
                    print(f"Aligning: {a_data} with {p_data}")
                    if aligned is None :
                        # Data compare combines to data frames
                        aligned = data_compare(devices[a_data], devices[p_data])
                        pairs["pair" + str(pair_count)] = aligned
                    else :
                        temp = data_compare(devices[a_data], devices[p_data])
                        pairs["pair" + str(pair_count)] = temp
                        aligned = pd.concat([aligned, temp], axis=0)
                    pair_count += 1
    return aligned, pairs

In [29]:
aligned_data, device_pairs = device_aligner(trial)

Aligning: acti1 with proxy1
Aligning: acti1 with proxy2
Aligning: acti2 with proxy1
Aligning: acti2 with proxy2
Aligning: acti3 with proxy1
Aligning: acti3 with proxy2
Aligning: acti4 with proxy1
Aligning: acti4 with proxy2
Aligning: acti5 with proxy1
Aligning: acti5 with proxy2


In [30]:
aligned_data

,Trial Number,Round Number,Speed,Actigraph ID,Actigraph Time,Include,Actigraph X,Actigraph Y,Actigraph Z,Proxy ID,Proxy Time,Reading #,Proxy X,Proxy Y,Proxy Z
0,2,1,3.2 Hz,77,2022-03-09 11:54:45.000,0,0.672,-0.484,-0.957,2458,2022-03-09 11:54:45,1,-12.0,-48.0,-1024.0
1,2,1,3.2 Hz,77,2022-03-09 11:54:45.034,0,0.063,-0.766,-0.98,2458,2022-03-09 11:54:45,2,-8.0,-56.0,-1036.0
2,2,1,3.2 Hz,77,2022-03-09 11:54:45.067,0,-0.367,-0.262,-1.027,2458,2022-03-09 11:54:45,3,-16.0,-48.0,-1032.0
3,2,1,3.2 Hz,77,2022-03-09 11:54:45.100,0,-0.297,-0.023,-1.051,2458,2022-03-09 11:54:45,4,-12.0,-56.0,-1032.0
4,2,1,3.2 Hz,77,2022-03-09 11:54:45.134,0,-0.289,-0.055,-1.043,2458,2022-03-09 11:54:45,5,-8.0,-52.0,-1032.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25195,2,1,0.6 Hz,20,2022-03-09 12:08:44.834,0,0.02,0.012,-1.004,NaN,NaT,NaN,NaN,NaN,NaN
25196,2,1,0.6 Hz,20,2022-03-09 12:08:44.867,0,0.02,0.012,-1.004,NaN,NaT,NaN,NaN,NaN,NaN
25197,2,1,0.6 Hz,20,2022-03-09 12:08:44.900,0,0.02,0.012,-1.004,NaN,NaT,NaN,NaN,NaN,NaN
25198,2,1,0.6 Hz,20,2022-03-09 12:08:44.934,0,0.02,0.012,-1.004,NaN,NaT,NaN,NaN,NaN,NaN


In [31]:
# This function strips the aligned data of it's microseconds.
# THis function also numbers the data sequentially
def data_order(dataframe):
    rows, columns = dataframe.shape
    num_list = [i for i in range(1, rows+1)]
    dataframe.insert(5, "Order", num_list, True)
    dataframe["Actigraph Time"] = dataframe["Actigraph Time"].apply(lambda x : x.replace(microsecond=0))
    return dataframe
aligned_data = data_order(aligned_data)


In [32]:
# path = "V:\\R01 - W4K\\2_Shaker project\\Devices Evaluated\\Trial90degree\\Processed_Data"
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data"
if device == 0:
    path = path + "\\Garmin\\Trial 0"+ str(trial_meta[0]) + "\\garmin_aligned.csv"
else :
    path = path + "\\Apple\\Trial " + str(trial_meta[0]) + "\\apple_alligned.csv"
aligned_data.to_csv(path, index=False)

In [33]:
device_pairs.keys()

dict_keys(['pair1', 'pair2', 'pair3', 'pair4', 'pair5', 'pair6', 'pair7', 'pair8', 'pair9', 'pair10'])

In [34]:
# This function takes as input a dataframe containing trial data for an aligned actigraph and proxy pair
# The output of this function is the inner minute data of each trial for this pair.
def inner_minute(data):
    outer_min, inner_min = data.groupby(by="Include")
    return inner_min[1]
    

In [35]:
def root_mean(dataframe, device):
    # Get shape of dataframe
    # print(f"Data Frame shape {dataframe.shape}")
    rows, columns = dataframe.shape
    # The convert to a numpy array
    data = dataframe.to_numpy()
    # Initialize output
    # Check which device the proxy is. Then divide the number of rows by the sample rate of the actigraph.
    # This should result in the ouput data frame having 420 rows, which corresponds to 7 minutes worth of readings
    # with a sample every second
    if device == 0 :
        out_rows = rows // 30
    else :
        out_rows = rows // 100
        
    # print(f"Out rows {out_rows}")
    data_out = np.zeros([out_rows, columns + 3], dtype="O")
    # out_row is the being written to in data_out
    out_row = 0
    # row is the row being read from in data
    row = 0
    prox_id = data[row,9]
    if np.isnan(prox_id):
        prox_id=data[row+1,9]
    while out_row < out_rows:
        #start represents the first row in data that begins a new second
        start = data[row,:]
        # Gets the time from row
        start_sec = start[4]
        # Calculates the next second
        end_sec = start_sec + timedelta(seconds=1)
        # intialize avg_sum,square_sum, and count which are used to calculate Avg and RMS
        # Each value in the lists correspond to an accel value Acti xyz and Proxy xyz
        avg = [0,0,0,0,0,0]
        squ = [0,0,0,0,0,0]
        acti_count = 0
        prox_count = 0
        # Iterates through seconds worth of data
        while row < rows and data[row, 4] < end_sec:
            # Add each reading to corresponding value in avg_sum
            avg[0] += data[row, 6]
            avg[1] += data[row, 7]
            avg[2] += data[row, 8]
            # Need to check if the proxy reading is a NaN value
            if not np.isnan(data[row, 12]):
                avg[3] += data[row, 12]
                avg[4] += data[row, 13]
                avg[5] += data[row, 14]
                
            # Add each reading to corresponding value in squ_sum
            squ[0] += data[row, 6]**2
            squ[1] += data[row, 7]**2
            squ[2] += data[row, 8]**2
            if not np.isnan(data[row, 12]):             
                squ[3] += data[row, 12]**2
                squ[4] += data[row, 13]**2
                squ[5] += data[row, 14]**2
                prox_count += 1
            
            acti_count += 1
            row += 1
        # Now to calculate average and rms
        for i in range(6):
            # Since the sampling rate of actigraph is higher than proxy need to use two different counts
            if i < 3:
                avg[i] = avg[i] / acti_count
                squ[i] = squ[i] / acti_count
            else :
                avg[i] = avg[i] / prox_count
                squ[i] = squ[i] / prox_count                
            
            squ[i] = math.sqrt(squ[i])
        # Add the needed values to the output row
        # Add trial # round # speed actigraph id and Time
        data_out[out_row, :5] = start[:5]
        # Add actigraph xyz mean
        data_out[out_row, 5:8] = avg[:3]
        # add actigraph xyz RMS
        data_out[out_row, 8:11] = squ[:3]
        # Add Proxy ID
        data_out[out_row, 11] = prox_id
        # Add Proxy xyz mean
        data_out[out_row, 12:15] = avg[3:]
        # add proxy xyz rms
        data_out[out_row, 15:] = squ[3:]
        
        out_row += 1
    headers = ['Trial Number', 'Round Number', 'Speed', 'Actigraph ID', 'Time', 'Actigraph Mean X', 'Actigraph Mean Y',
              'Actigraph Mean Z', 'Actigraph RMS X', 'Actigraph RMS Y', 'Actigraph RMS Z', 'Proxy ID', 'Proxy Mean X',
              'Proxy Mean Y', 'Proxy Mean Z', 'Proxy RMS X', 'Proxy RMS Y', 'Proxy RMS Z']
    df_out = pd.DataFrame(data_out, columns = headers)
    return df_out
        
    

In [36]:
# This function pulls the inner minute of each device pair, calculates the RMS and average values at a second level
# It takes as input a dictionary of dataframes where each dataframe corresponds to 1 actigraph and proxy aligned trial data
# and the start time of each trial
# It outputs a single data frame that includes each actigraph proxy pair, where the data is aggregated to the second level 
# And the RMS is calculated for each second.
def aggregate(pairs, device):
    output_df = None
    # Iterate through pairs dictionary
    for pair in pairs:
        # First the inner minute of each trial needs to be extracted from each data frame:
        inner_data = inner_minute(pairs[pair])
        # Calculate the RMS and average of acceleration data
        rms = root_mean(inner_data, device)
        if output_df is None:
            output_df = rms
        else:
            output_df = pd.concat([output_df, rms], axis=0)
        
    return output_df

In [37]:
rms_data = aggregate(device_pairs,device)
rms_data

,Trial Number,Round Number,Speed,Actigraph ID,Time,Actigraph Mean X,Actigraph Mean Y,Actigraph Mean Z,Actigraph RMS X,Actigraph RMS Y,Actigraph RMS Z,Proxy ID,Proxy Mean X,Proxy Mean Y,Proxy Mean Z,Proxy RMS X,Proxy RMS Y,Proxy RMS Z
0,2,1,3.2 Hz,77,2022-03-09 11:55:15,-0.013933,0.050333,-1.025033,0.334988,0.373858,1.026387,2458,20.16,-88.16,-1033.6,380.831931,335.341259,1034.289553
1,2,1,3.2 Hz,77,2022-03-09 11:55:16,0.0299,-0.0241,-1.0259,0.330973,0.360499,1.027015,2458,-37.76,-25.6,-1029.12,371.501602,313.730585,1029.591608
2,2,1,3.2 Hz,77,2022-03-09 11:55:17,-0.034333,0.036133,-1.028533,0.339589,0.369586,1.029808,2458,11.52,-99.52,-1032.0,376.428692,336.070469,1032.603854
3,2,1,3.2 Hz,77,2022-03-09 11:55:18,0.0425,-0.009033,-1.020267,0.339304,0.35861,1.021434,2458,-30.56,-22.56,-1033.12,388.863369,321.242587,1033.657893
4,2,1,3.2 Hz,77,2022-03-09 11:55:19,-0.041833,0.0112,-1.025867,0.333526,0.365347,1.026922,2458,7.68,-94.72,-1027.52,355.675358,330.590744,1027.948637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2,1,0.6 Hz,20,2022-03-09 12:08:10,0.02,0.012,-1.004,0.02,0.012,1.004,2428,-3.84,-26.56,-967.84,10.673331,27.387588,967.862139
416,2,1,0.6 Hz,20,2022-03-09 12:08:11,0.02,0.012,-1.004,0.02,0.012,1.004,2428,-11.04,-35.36,-968.96,14.945233,36.035538,969.02227
417,2,1,0.6 Hz,20,2022-03-09 12:08:12,0.02,0.012,-1.004,0.02,0.012,1.004,2428,-1.76,-25.76,-970.4,9.015542,26.761166,970.41385
418,2,1,0.6 Hz,20,2022-03-09 12:08:13,0.02,0.012,-1.004,0.02,0.012,1.004,2428,-12.0,-31.84,-966.24,14.59863,32.623918,966.31291


In [38]:
# path = "V:\\R01 - W4K\\2_Shaker project\\Devices Evaluated\\Trial90degree\\Processed_Data"
path = "C:\\Users\\Nick\\Watch_Extraction\\Shaker_Table\\Data"
if device == 0:
    path = path + "\\Garmin\\Trial 0"+ str(trial_meta[0]) + "\\garmin_rms.csv"
else :
    path = path + "\\Apple\\Trial " + str(trial_meta[0]) + "\\apple_rms.csv"
rms_data.to_csv(path, index=False)

In [360]:
trial_meta[0]

6